# Installation des dépendances

Pour pouvoir exécuter ce Notebook, l'environnement de développement doit être bien configuré.

<div class="alert alert-info">
    Pour ce Notebook, on pourra choisir le kernel <code>conda_tensorflow2_p310</code> sur SageMaker.
</div>

In [ ]:
!pip install spacy transformers datasets sentencepiece seaborn pandas -q

De même, nous allons utiliser un jeu de données supplémentaire de HuggingFace pour enrichir notre base.

In [ ]:
!git lfs install
!git clone https://huggingface.co/datasets/Paul/hatecheck-french

Enfin, spaCy nous permettra de faire des opérations de traitement du texte adapté à la langue française.

In [ ]:
!python -m spacy download fr_core_news_sm

# Analyse exploratoire des données

Pour cette analyse exploiratoire, le dataset `mlma.csv` doit être présent dans l'environnement de développement, situé dans le même dossier que ce Notebook.

In [ ]:
!mkdir -p data/
!wget -q https://blent-keskia.s3.eu-west-3.amazonaws.com/poc/cyberharcelement/french_tweets.csv.zip -O data/french_tweets.csv.zip
!wget -q https://blent-keskia.s3.eu-west-3.amazonaws.com/poc/cyberharcelement/mlma.csv -O data/mlma.csv
!unzip -o data/french_tweets.csv.zip -d data

In [ ]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sns.set()

data = pd.read_csv("data/mlma.csv")
tweets = pd.read_csv("data/french_tweets.csv")

De même, nous chargeons le jeu de données HateCheck téléchargé depuis HuggingFace.

In [ ]:
hatecheck = pd.read_csv("hatecheck-french/test.csv")

Regardons à quoi ressemblent les deux jeux de données.

In [ ]:
data.head()

In [ ]:
hatecheck.head()

La colonne `sentiment` du premier jeu de données indique si le Tweet est normal, offensif ou encore haineux.

In [ ]:
data["sentiment"].unique()

Il y a potentiellement des modalités qui se chevauchent : nous pourrions être amenés à en fusionner plusieurs entre-elles.

In [ ]:
hatecheck["label_gold"].unique()

Le deuxième jeu de données est quant à lui binaire sur le sentiment.

In [ ]:
data["group"].unique()

Ce qui est intéressant avec les deux jeux de données, c'est que nous avons également à disposition le groupe d'individus visé par le commentaire.

In [ ]:
hatecheck["target_ident"].unique()

Essayons de visualiser quelques lignes.

### ➡️ À toi de jouer

Afficher quelques lignes donc le sentiment est `fearful_normal` pour la dataset MLMA.

In [ ]:
# TODO


### ➡️ À toi de jouer

Afficher quelques lignes donc le sentiment est `normal` pour la dataset MLMA.

In [ ]:
# TODO
# ...

Ce qui est important pour ce POC, c'est de **différencier ce qui relève de la plaisanterie dans un groupe d'amis d'un harcèlement haineux**. Pour cela, la **notion de contexte sera donc très importante**.

Dans ces deux exemples précédents, nous allons considérer les sentiments `normal` et `fearful_normal` comme `non-hateful`, de sorte à harmoniser les labels entre les deux jeux de données.

### ➡️ À toi de jouer

Sur la dataset MLMA, transformer tous les sentiments `normal` et `fearful_normal` en `non-hateful`.

In [ ]:
# TODO
# ...

Maintenant, nous allons **concaténer tous les datasets en un seul**.

In [ ]:
from sklearn.utils import shuffle

# df représente notre DataFrame final du traitement de données
df1 = pd.DataFrame()
df1["Text"] = data["tweet"]
df1["Group"] = data["group"]
df1["Type"] = data["sentiment"]

df2 = pd.DataFrame()
df2["Text"] = hatecheck["test_case"]
df2["Group"] = hatecheck["target_ident"]
df2["Type"] = hatecheck["label_gold"]

# On ajoute des Tweets random non offensants
df3 = pd.DataFrame()
n_tweets = 10000
df3["Text"] = tweets.sample(n=n_tweets)["text"]
df3["Group"] = "other"
df3["Type"] = "non-hateful"

df = pd.concat([df1, df2, df3], axis=0)
df = shuffle(df)

Dans notre cas, nous allons rester sur de la classification binaire.

### ➡️ À toi de jouer

Dans le dataset `df`, remplacer tous les types **différents de `non-hateful`** par `hateful`.

In [ ]:
# TODO
# ...
df

Vérifions bien que nous n'ayons que deux classes.

In [ ]:
df["Type"].unique()

Vérifions également les groupes d'individus.

In [ ]:
df["Group"].unique()

Là-aussi pour simplifier l'entraînement du modèle, nous allons fusionner certains groupes.

In [ ]:
df["Group"].loc[(df["Group"] == "gay people")] = "gay"
df["Group"].loc[(df["Group"] == "trans people")] = "gay"
df["Group"].loc[(df["Group"] == "black people")] = "african_descent"
df["Group"].loc[(df["Group"] == "Muslims")] = "muslims"
df["Group"].loc[(df["Group"] == "disabled people")] = "special_needs"
df["Group"].loc[(df["Group"] == "refugees")] = "immigrants"
df["Group"].loc[(df["Group"] == "individual")] = "other"

Regardons à nouveau les groupes uniques.

In [ ]:
df["Group"].unique()

In [ ]:
sns.countplot(x=df["Type"])
plt.show()

Visuellement, nous avons une sur-représentation de commentaires haineux : il faudrait potentiellement enrichir le jeu de données avec des commentaires non haineux.

### ➡️ À toi de jouer

Afficher le nombre de lignes de données pour chaque communauté/groupe d'individus.

In [ ]:
# TODO
# ...

Concernant les groupes d'individus, la plupart sont plutôt équilibrés, même si les 4 derniers groupes semblent être moins représentés.

### ➡️ À toi de jouer

Afficher le nombre de lignes de données `hateful` et `non-hateful` pour chaque communauté/groupe d'individus.

In [ ]:
# TODO
# ...